# Abdolazizsalimi980292626Ex6


## CIFAR-10 Dataset

The CIFAR-10 dataset consists of 60,000 32x32 colour images divided into 10 classes, with 6,000 images per class. The dataset is further divided into training and test sets, with 50,000 training images and 10,000 test images.




### Given an image, can we predict the correct class of this image?

The images are very small (32x32) and by visualizing them you will notice how difficult it is to distinguish them even for a human.

In this notebook we are going to build a CNN model that can classify images of various objects. We have 10 class of images:

1. Airplane
2. Automobile
3. Bird
4. Cat
5. Deer
6. Dog
7. Frog
8. Horse
9. Ship
10. Truck

### Training and Test Batches

The dataset is organized into five training batches and one test batch, each containing 10,000 images. The test batch comprises 1,000 randomly-selected images from each class, resulting in a total of 10,000 test images. 

In contrast, the training batches contain the remaining images and are organized in random order. It is important to note that some training batches may have an uneven distribution of images across classes, with more images from one class than another. However, collectively, the training batches contain exactly 5,000 images from each class.

This division allows researchers and practitioners to train machine learning models on the training set and evaluate their performance on the test set, providing a standardized benchmark for image classification tasks.



In [106]:
import importlib
importlib.invalidate_caches()
import tensorflow as tf

In [107]:
tf.debugging.set_log_device_placement(True)
gpus = tf.config.list_physical_devices('GPU')

In [108]:
gpus

[]

In [109]:

if gpus:
  # Restrict TensorFlow to only use the first GPU
  try:
    tf.config.set_visible_devices(gpus[0], 'GPU')
    logical_gpus = tf.config.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPU")
  except RuntimeError as e:
    # Visible devices must be set before GPUs have been initialized
    print(e)

In [ ]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:

from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical

In [ ]:

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPool2D, Flatten, Dropout, BatchNormalization

from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.metrics import classification_report, confusion_matrix

In [ ]:
# batch_size = 1024  # The default batch size of keras.
# num_classes = 10  # Number of class for the dataset
# epochs = 50

In [ ]:
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

print(f"X_train shape: {X_train.shape}")
print(f"y_train shape: {y_train.shape}")
print(f"X_test shape: {X_test.shape}")
print(f"y_test shape: {y_test.shape}")

In [ ]:
# Define the labels of the dataset
labels = ['airplane', 'automobile', 'bird', 'cat', 'deer', 
          'dog', 'frog', 'horse', 'ship', 'truck']

# Let's view more images in a grid format
# Define the dimensions of the plot grid 
W_grid = 4
L_grid = 4



fig, axes = plt.subplots(L_grid, W_grid, figsize = (16 ,10))

axes = axes.ravel() # flaten the 15 x 15 matrix into 225 array

n_train = len(X_train) # get the length of the train dataset

# Select a random number from 0 to n_train
for i in np.arange(0, W_grid * L_grid): # create evenly spaces variables 

    # Select a random number
    index = np.random.randint(0, n_train)
    # read and display an image with the selected index    
    axes[i].imshow(X_train[index,1:])
    label_index = int(y_train[index])
    axes[i].set_title(labels[label_index], fontsize = 8)
    axes[i].axis('off')

plt.subplots_adjust(hspace=0.4)

In [ ]:
classes_name = ['Airplane', 'Automobile', 'Bird', 'Cat', 'Deer', 'Dog', 'Frog', 'Horse', 'Ship', 'Truck']

fig, axs = plt.subplots(1, 2, figsize=(15, 5))

# Count plot for training set
sns.countplot(y_train.ravel(), ax=axs[0])
axs[0].set_title('Distribution of Training Data')
axs[0].set_xlabel('Classes')
axs[0].set_ylabel('Count')
axs[0].set_xticklabels(classes_name)

# Count plot for testing set
sns.countplot(y_test.ravel(), ax=axs[1])
axs[1].set_title('Distribution of Testing Data')
axs[1].set_xlabel('Classes')
axs[1].set_ylabel('Count')
axs[1].set_xticklabels(classes_name)

plt.tight_layout()
plt.show()

In [ ]:

# Transform target variable into one-hot-encoding
y_cat_train = to_categorical(y_train, 10)
y_cat_test = to_categorical(y_test, 10)

## Designing The  Neural Network:

In [ ]:
model = Sequential()

# Convolutional Layer
model.add(Conv2D(filters=32, kernel_size=(3, 3), input_shape=(32, 32, 3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(Conv2D(filters=32, kernel_size=(3, 3), input_shape=(32, 32, 3), activation='relu', padding='same'))
model.add(BatchNormalization())
# Pooling layer
model.add(MaxPool2D(pool_size=(2, 2)))
# Dropout layers
model.add(Dropout(0.25))

model.add(Conv2D(filters=64, kernel_size=(3, 3), input_shape=(32, 32, 3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(Conv2D(filters=64, kernel_size=(3, 3), input_shape=(32, 32, 3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(filters=128, kernel_size=(3, 3), input_shape=(32, 32, 3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(Conv2D(filters=128, kernel_size=(3, 3), input_shape=(32, 32, 3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
# model.add(Dropout(0.2))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(10, activation='softmax'))

In [ ]:
model.summary()

In [ ]:
import visualkeras
visualkeras.layered_view(model, scale_xy=10, legend=True)

In [ ]:


x_train = X_train.astype('float32')
x_test = X_test.astype('float32')

# Normalizing the input image
import numpy as np

# Compute mean and standard deviation of the training set
mean = np.mean(x_train, axis=(0, 1, 2))
std = np.std(x_train, axis=(0, 1, 2))


# Standardize the data
x_train = (x_train - mean) / std
x_test = (x_test - mean) / std




METRICS = [
    'accuracy',
    tf.keras.metrics.Precision(name='precision'),
    tf.keras.metrics.Recall(name='recall')
]

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=METRICS)

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
early_stop = EarlyStopping(monitor='val_loss', patience=2)
batch_size = 32
data_generator = ImageDataGenerator(width_shift_range=0.1, height_shift_range=0.1, horizontal_flip=True)
train_generator = data_generator.flow(x_train, y_cat_train, batch_size)
steps_per_epoch = x_train.shape[0] // batch_size

In [ ]:
model.fit(train_generator, 
              epochs=36,
              steps_per_epoch=steps_per_epoch,
              validation_data=(x_test, y_cat_test), 
             )

In [ ]:
history = model.history.history
df=pd.DataFrame(history)

In [ ]:
df[['accuracy','val_accuracy']].plot()

In [ ]:
df[['loss','val_loss']].plot()

In [ ]:
# Score trained model.
model.evaluate(x_test, y_cat_test)


In [ ]:
predictions=np.argmax(model.predict(x_test),1)

In [ ]:
print(classification_report(y_test,predictions))

In [ ]:
y_pred=model.predict(x_test)
y_pred

In [ ]:
y_classes=[np.argmax(element) for element in y_pred]

In [ ]:
def predict_which_category(x, y, index):
    print("predict to be", classes_name[y_classes[index]])
    plt.figure(figsize = (15,2))
    plt.imshow(x[index])

In [ ]:
predict_which_category(x_test,y_test,1)

In [ ]:
predict_which_category(x_test,y_test,4)

In [ ]:
model.save('best-CNN-model.h5')

In [ ]:
from tensorflow.keras.models import load_model
from tensorflow.keras.optimizers import Adam

In [ ]:

model_path = r'./best-CNN-model.h5'
model_path2 = r'./best_CNN-model.h5'

# Load the model with the custom optimizer

model = load_model(model_path)
model = load_model(model_path2)